In [1]:
import os
from pathlib import Path
import pandas as pd

os.environ["OPENBLAS_NUM_THREADS"] = "1"

# --- CẤU HÌNH ---
INPUT_CSV = Path("Dataset/EE2_K62_K63_K64_.csv")
OUTPUT_XLSX = Path("Data_clean/Data_subject_pivot.xlsx")
REQUIRED_COURSES = [
    "Tin học đại cương",
    "Giải tích I",
    "Giải tích II",
    "Giải tích III",
    "Đại số",
    "Xác suất thống kê",
    "Phương pháp tính và MATLAB",
    "Vật lý đại cương I",
    "Vật lý đại cương II",
    "Vật lý đại cương III",
    "Nhập môn ngành Kỹ thuật điều khiển và Tự động hóa",
    "Tín hiệu và hệ thống",
    "Lý thuyết mạch điện I",
    "Lý thuyết mạch điện II",
    "Trường điện từ",
    "Điện tử tương tự",
    "Thiết kế hệ thống số",
    "Kỹ thuật đo lường",
    "Máy điện I",
    "Lý thuyết điều khiển tuyến tính",
    "Điện tử công suất",
    "Hệ thống cung cấp điện (BTL)",
    "Vi xử lý",
    "Kỹ thuật lập trình",
    "Truyền động điện",
    "Đồ án I",
    "Đồ án II"
]

GRADE_MAPPING = {
    'A+': 4.0, 'A': 4.0, 'B+': 3.5, 'B': 3.0,
    'C+': 2.5, 'C': 2.0, 'D+': 1.5, 'D': 1.0
}

# --- HÀM CHÍNH ---
def process_grades(input_csv: Path, output_xlsx: Path):
    print(f"🔍 Đang đọc dữ liệu từ: {input_csv}")
    df = pd.read_csv(input_csv)

    # Chọn cột cần thiết
    needed_columns = ["EncryptedID", "Course ID", "Course Name", "Final Grade"]
    missing_cols = [c for c in needed_columns if c not in df.columns]
    if missing_cols:
        raise ValueError(f"Các cột sau thiếu trong file nguồn: {missing_cols}")
    df_cleaned = df[needed_columns].copy()
    
    # # Chuẩn hóa tên môn 
    # df_cleaned["Course Name"] = df_cleaned["Course Name"].replace({
    #     "Nhập môn ngành Điện": "Nhập môn ngành Kỹ thuật điều khiển và Tự động hóa"
    # })

    # Map điểm chữ sang số
    df_cleaned["Final Grade Numeric"] = df_cleaned["Final Grade"].map(GRADE_MAPPING)

    # Báo nếu có grade không hợp lệ
    invalid_grades = df_cleaned["Final Grade"].isna() & df_cleaned["Final Grade Numeric"].isna()
    if invalid_grades.any():
        count_invalid = invalid_grades.sum()
        print(f"⚠️ Có {count_invalid} dòng có 'Final Grade' không nằm trong mapping và sẽ bị loại (ví dụ: {df_cleaned.loc[invalid_grades, 'Final Grade'].unique()[:5]})")

    # Loại bỏ NaN sau khi map (chỉ giữ những grade hợp lệ)
    df_cleaned = df_cleaned.dropna(subset=["Final Grade Numeric"])

    # Lọc các môn cần thiết (nếu Course Name trùng)
    df_filtered = df_cleaned[df_cleaned["Course Name"].isin(REQUIRED_COURSES)].copy()
    print(f"✅ Sau khi lọc theo danh sách môn: còn {len(df_filtered)} dòng.")

    

    # Lấy điểm tốt nhất của mỗi sinh viên cho mỗi môn
    df_best = (
        df_filtered
        .groupby(["EncryptedID", "Course ID", "Course Name"], dropna=False)["Final Grade Numeric"]
        .max()
        .reset_index()
    )
    
    # Pivot dùng best scores
    pivot_df = df_best.pivot_table(
        index="EncryptedID",
        columns="Course Name",
        values="Final Grade Numeric",
        aggfunc="first"  # đã là max rồi
    )

    # Đảm bảo thứ tự cột theo REQUIRED_COURSES, nhưng chỉ lấy những course thực sự có trong dữ liệu
    existing_cols = [c for c in REQUIRED_COURSES if c in pivot_df.columns]
    pivot_df = pivot_df.reindex(columns=existing_cols)
    pivot_df = pivot_df.reset_index()

    

    # Tạo thư mục đầu ra nếu chưa có
    output_xlsx.parent.mkdir(parents=True, exist_ok=True)
    pivot_df.to_excel(output_xlsx, index=False)
    print(f"✅ Đã lưu bảng xoay (pivot) vào: {output_xlsx}")

if __name__ == "__main__":
    process_grades(INPUT_CSV, OUTPUT_XLSX)


🔍 Đang đọc dữ liệu từ: Dataset\EE2_K62_K63_K64_.csv
✅ Sau khi lọc theo danh sách môn: còn 37127 dòng.
✅ Đã lưu bảng xoay (pivot) vào: Data_clean\Data_subject_pivot.xlsx


In [2]:
import os
from pathlib import Path
import pandas as pd

os.environ["OPENBLAS_NUM_THREADS"] = "1"

INPUT = Path("Data_clean/Data_subject_pivot.xlsx")
OUTPUT = Path("Data_clean/Data_subject_complete.xlsx")

REQUIRED_COURSES = [
    "Tin học đại cương",
    "Giải tích I",
    "Giải tích II",
    "Giải tích III",
    "Đại số",
    "Xác suất thống kê",
    "Phương pháp tính và MATLAB",
    "Vật lý đại cương I",
    "Vật lý đại cương II",
    "Vật lý đại cương III",
    "Nhập môn ngành Kỹ thuật điều khiển và Tự động hóa",
    "Tín hiệu và hệ thống",
    "Lý thuyết mạch điện I",
    "Lý thuyết mạch điện II",
    "Trường điện từ",
    "Điện tử tương tự",
    "Thiết kế hệ thống số",
    "Kỹ thuật đo lường",
    "Máy điện I",
    "Lý thuyết điều khiển tuyến tính",
    "Điện tử công suất",
    "Hệ thống cung cấp điện (BTL)",
    "Vi xử lý",
    "Kỹ thuật lập trình",
    "Truyền động điện",
    "Đồ án I",
    "Đồ án II"
]


# Đọc dữ liệu
df = pd.read_excel(INPUT)

# Giữ chỉ EncryptedID và những môn thực sự có trong file
available_courses = [c for c in REQUIRED_COURSES if c in df.columns]
keep_cols = ["EncryptedID"] + available_courses
df = df[keep_cols].copy()

# Lọc chỉ sinh viên có đầy đủ điểm (không có NaN ở tất cả các môn)
df_complete = df.dropna(subset=available_courses, how="any")  # tương đương với: ~df[available_courses].isna().any(axis=1)

df_complete = df_complete.sample(frac=1, random_state=42).reset_index(drop=True)
n = len(df_complete)
n_train = int(n * 0.6)
n_val = int(n * 0.2)
n_test = n - n_train - n_val

split_col = (
    ["train"] * n_train + 
    ["val"] * n_val + 
    ["test"] * n_test
)

df_complete["split"] = split_col



# Ghi ra file
OUTPUT.parent.mkdir(parents=True, exist_ok=True)
df_complete.to_excel(OUTPUT, index=False)
print(f"✅ Đã xuất sinh viên đủ tất cả môn (chỉ ID + môn) vào: {OUTPUT}")


✅ Đã xuất sinh viên đủ tất cả môn (chỉ ID + môn) vào: Data_clean\Data_subject_complete.xlsx
